In [59]:
import os
import re

def clean_text(text):
    # Loại bỏ các ký tự đặc biệt như ô trống hoặc các ký tự không mong muốn
    return re.sub(r'[\x00-\x1F\x7F]', '', text).strip()

def split_txt_into_chunks(text, chunk_size=1024):
    # Tách văn bản thành các đoạn nhỏ hơn với kích thước không vượt quá chunk_size
    chunks = []
    while len(text) > chunk_size:
        # Tìm điểm chia tách gần nhất, ưu tiên chia tại dấu phân cách 'Điều'
        split_point = max(text.rfind('Điều', 0, chunk_size), text.rfind(' ', 0, chunk_size))
        if split_point == -1:
            split_point = chunk_size
        chunks.append(text[:split_point].strip())
        text = text[split_point:].strip()
    chunks.append(text)
    return chunks

def split_txt_by_rules_with_header(input_file, output_folder):
    # Tạo thư mục đích nếu chưa tồn tại
    os.makedirs(output_folder, exist_ok=True)
    
    # Đọc toàn bộ nội dung của file .txt
    with open(input_file, 'r', encoding='utf-8') as file:
        full_text = [clean_text(line) for line in file.readlines() if clean_text(line) != '']
    
    # Lấy thông tin về "Luật số", "Ngày ban hành", và "Tên bộ luật"
    law_number = ""
    date_info = ""
    law_title = ""
    
    for paragraph in full_text:
        if "Luật số" in paragraph:
            law_number = paragraph
        elif "ngày" in paragraph and "tháng" in paragraph and "năm" in paragraph:
            date_info = paragraph
        elif paragraph.isupper():  # Giả định rằng tiêu đề luật là viết hoa toàn bộ
            law_title = paragraph
            break
    
    # Tách văn bản thành các đoạn theo từ khóa "Điều"
    dieu_sections = []
    current_section = []
    current_condition = ""
    
    for paragraph in full_text:
        if paragraph.startswith("Điều"):
            if current_section:
                dieu_sections.append((current_condition, "\n".join(current_section)))
                current_section = []
            current_condition = paragraph
        current_section.append(paragraph)
    if current_section:
        dieu_sections.append((current_condition, "\n".join(current_section)))
    
    # Xử lý và ghi từng phần vào các file riêng biệt với thông tin bổ sung
    file_count = 0
    for i, (condition, section_text) in enumerate(dieu_sections):
        # Nếu nội dung không vượt quá kích thước tối đa, không cần chia nhỏ
        if len(section_text) <= 1024:
            output_file = os.path.join(output_folder, f"Dieu_{i+1}.txt")
            with open(output_file, 'w', encoding='utf-8') as file:
                # Thêm thông tin vào đầu file
                file.write(f"{law_number}\n")
                file.write(f"{date_info}\n\n")
                file.write(f"{law_title}\n\n")
                # Thêm phần nội dung của Điều
                file.write(section_text)
            file_count += 1
        else:
            # Nếu nội dung vượt quá kích thước tối đa, chia nhỏ và ghi vào nhiều file
            chunks = split_txt_into_chunks(section_text, chunk_size=1024)
            for j, chunk in enumerate(chunks):
                output_file = os.path.join(output_folder, f"Dieu_{i+1}_{j+1}.txt")
                with open(output_file, 'w', encoding='utf-8') as file:
                    # Thêm thông tin vào đầu file
                    if j > 0:  # Chỉ bổ sung thông tin "Điều" vào file thứ 2 trở đi
                        file.write(f"{law_number}\n")
                        file.write(f"{date_info}\n\n")
                        file.write(f"{law_title}\n\n")
                        file.write(f"{condition}\n")
                    # Thêm phần nội dung của Điều
                    file.write(chunk)
                
                file_count += 1
    
    print(f"Đã tách file {input_file} thành {file_count} phần trong thư mục {output_folder}.")



TÁCH THEO PDF

In [61]:
import os
import re
import fitz  # PyMuPDF

def clean_text(text):
    # Loại bỏ các ký tự đặc biệt như ô trống hoặc các ký tự không mong muốn
    return re.sub(r'[\x00-\x1F\x7F]', '', text).strip()

def split_text_into_chunks(text, chunk_size=1024):
    # Tách văn bản thành các đoạn nhỏ hơn với kích thước không vượt quá chunk_size
    chunks = []
    while len(text) > chunk_size:
        # Tìm điểm chia tách gần nhất, ưu tiên chia tại dấu phân cách 'Điều'
        split_point = max(text.rfind('Điều', 0, chunk_size), text.rfind(' ', 0, chunk_size))
        if split_point == -1:
            split_point = chunk_size
        chunks.append(text[:split_point].strip())
        text = text[split_point:].strip()
    chunks.append(text)
    return chunks

def split_pdf_by_rules_with_header(input_file, output_folder):
    # Tạo thư mục đích nếu chưa tồn tại
    os.makedirs(output_folder, exist_ok=True)
    
    # Mở file PDF
    doc = fitz.open(input_file)
    
    # Đọc toàn bộ nội dung của file PDF
    full_text = []
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text = page.get_text()
        if text:
            full_text.extend([clean_text(line) for line in text.split('\n') if clean_text(line) != ''])
    
    # Đóng file PDF
    doc.close()
    
    # Lấy thông tin về "Luật số", "Ngày ban hành", và "Tên bộ luật"
    law_number = ""
    date_info = ""
    law_title = ""
    
    for paragraph in full_text:
        if "Luật số" in paragraph:
            law_number = paragraph
        elif "ngày" in paragraph and "tháng" in paragraph and "năm" in paragraph:
            date_info = paragraph
        elif paragraph.isupper():  # Giả định rằng tiêu đề luật là viết hoa toàn bộ
            law_title = paragraph
            break
    
    # Tách văn bản thành các đoạn theo từ khóa "Điều"
    dieu_sections = []
    current_section = []
    current_condition = ""
    
    for paragraph in full_text:
        if paragraph.startswith("Điều"):
            if current_section:
                dieu_sections.append((current_condition, "\n".join(current_section)))
                current_section = []
            current_condition = paragraph
        current_section.append(paragraph)
    if current_section:
        dieu_sections.append((current_condition, "\n".join(current_section)))
    
    # Xử lý và ghi từng phần vào các file riêng biệt với thông tin bổ sung
    file_count = 0
    for i, (condition, section_text) in enumerate(dieu_sections):
        # Nếu nội dung không vượt quá kích thước tối đa, không cần chia nhỏ
        if len(section_text) <= 1024:
            output_file = os.path.join(output_folder, f"Dieu_{i+1}.txt")
            with open(output_file, 'w', encoding='utf-8') as file:
                # Thêm thông tin vào đầu file
                file.write(f"{law_number}\n")
                file.write(f"{date_info}\n\n")
                file.write(f"{law_title}\n\n")
                # Thêm phần nội dung của Điều
                file.write(section_text)
            file_count += 1
        else:
            # Nếu nội dung vượt quá kích thước tối đa, chia nhỏ và ghi vào nhiều file
            chunks = split_text_into_chunks(section_text, chunk_size=1024)
            for j, chunk in enumerate(chunks):
                output_file = os.path.join(output_folder, f"Dieu_{i+1}_{j+1}.txt")
                with open(output_file, 'w', encoding='utf-8') as file:
                    # Thêm thông tin vào đầu file
                    if j > 0:  # Chỉ bổ sung thông tin "Điều" vào file thứ 2 trở đi
                        file.write(f"{law_number}\n")
                        file.write(f"{date_info}\n\n")
                        file.write(f"{law_title}\n\n")
                        file.write(f"{condition}\n")
                    else:  # File đầu tiên không cần bổ sung thông tin "Điều"
                        file.write(f"{law_number}\n")
                        file.write(f"{date_info}\n\n")
                        file.write(f"{law_title}\n\n")
                    # Thêm phần nội dung của Điều
                    file.write(chunk)
                
                file_count += 1
    
    print(f"Đã tách file {input_file} thành {file_count} phần trong thư mục {output_folder}.")


TÁCH THEO DOC

In [12]:
import os
import pythoncom
from win32com.client import Dispatch
import re

def clean_text(text):
    # Loại bỏ các ký tự đặc biệt như ô trống hoặc các ký tự không mong muốn
    return re.sub(r'[\x00-\x1F\x7F]', '', text).strip()

def split_text_into_chunks(text, chunk_size=1024):
    # Tách văn bản thành các đoạn nhỏ hơn với kích thước không vượt quá chunk_size
    chunks = []
    while len(text) > chunk_size:
        # Tìm điểm chia tách gần nhất, ưu tiên chia tại dấu phân cách 'Điều'
        split_point = max(text.rfind('Điều', 0, chunk_size), text.rfind(' ', 0, chunk_size))
        if split_point == -1:
            split_point = chunk_size
        chunks.append(text[:split_point].strip())
        text = text[split_point:].strip()
    chunks.append(text)
    return chunks

def split_doc_by_rules_with_header(input_file, output_folder):
    # Tạo thư mục đích nếu chưa tồn tại
    os.makedirs(output_folder, exist_ok=True)
    
    # Khởi tạo COM object để tương tác với Microsoft Word
    pythoncom.CoInitialize()
    word = Dispatch("Word.Application")
    doc = word.Documents.Open(input_file)
    
    # Đọc toàn bộ nội dung của file
    full_text = [clean_text(para.Range.Text) for para in doc.Paragraphs if clean_text(para.Range.Text) != '']
    
    # Đóng file và Word application
    doc.Close(False)  # Không lưu thay đổi
    word.Quit()
    
    # Lấy thông tin về "Luật số", "Ngày ban hành", và "Tên bộ luật"
    law_number = ""
    date_info = ""
    law_title = ""
    
    for paragraph in full_text:
        if "Luật số" in paragraph:
            law_number = paragraph
        elif "ngày" in paragraph and "tháng" in paragraph and "năm" in paragraph:
            date_info = paragraph
        elif paragraph.isupper():  # Giả định rằng tiêu đề luật là viết hoa toàn bộ
            law_title = paragraph
            break
    
    # Tách văn bản thành các đoạn theo từ khóa "Điều"
    dieu_sections = []
    current_section = []
    current_condition = ""
    
    for paragraph in full_text:
        if paragraph.startswith("Điều"):
            if current_section:
                dieu_sections.append((current_condition, "\n".join(current_section)))
                current_section = []
            current_condition = paragraph
        current_section.append(paragraph)
    if current_section:
        dieu_sections.append((current_condition, "\n".join(current_section)))
    
    # Xử lý và ghi từng phần vào các file riêng biệt với thông tin bổ sung
    file_count = 0
    for i, (condition, section_text) in enumerate(dieu_sections):
        # Nếu nội dung không vượt quá kích thước tối đa, không cần chia nhỏ
        if len(section_text) <= 1024:
            output_file = os.path.join(output_folder, f"Dieu_{i+1}.txt")
            with open(output_file, 'w', encoding='utf-8') as file:
                # Thêm thông tin vào đầu file
                file.write(f"{law_number}\n")
                file.write(f"{date_info}\n\n")
                file.write(f"{law_title}\n\n")
                # Thêm phần nội dung của Điều
                file.write(section_text)
            file_count += 1
        else:
            # Nếu nội dung vượt quá kích thước tối đa, chia nhỏ và ghi vào nhiều file
            chunks = split_text_into_chunks(section_text, chunk_size=1024)
            for j, chunk in enumerate(chunks):
                output_file = os.path.join(output_folder, f"Dieu_{i+1}_{j+1}.txt")
                with open(output_file, 'w', encoding='utf-8') as file:
                    # Thêm thông tin vào đầu file
                    if j > 0:  # Chỉ bổ sung thông tin "Điều" vào file thứ 2 trở đi
                        file.write(f"{law_number}\n")
                        file.write(f"{date_info}\n\n")
                        file.write(f"{law_title}\n\n")
                        file.write(f"{condition}\n")
                    else:  # File đầu tiên không cần bổ sung thông tin "Điều"
                        file.write(f"{law_number}\n")
                        file.write(f"{date_info}\n\n")
                        file.write(f"{law_title}\n\n")
                    # Thêm phần nội dung của Điều
                    file.write(chunk)
                
                file_count += 1
    
    print(f"Đã tách file {input_file} thành {file_count} phần trong thư mục {output_folder}.")


TÁCH THEO DOCX

In [54]:
import os
import re
from docx import Document

def clean_text(text):
    # Loại bỏ các ký tự đặc biệt như ô trống hoặc các ký tự không mong muốn
    return re.sub(r'[\x00-\x1F\x7F]', '', text).strip()

def split_text_into_chunks(text, chunk_size=1024):
    # Tách văn bản thành các đoạn nhỏ hơn với kích thước không vượt quá chunk_size
    chunks = []
    while len(text) > chunk_size:
        # Tìm điểm chia tách gần nhất, ưu tiên chia tại dấu phân cách 'Điều'
        split_point = max(text.rfind('Điều', 0, chunk_size), text.rfind(' ', 0, chunk_size))
        if split_point == -1:
            split_point = chunk_size
        chunks.append(text[:split_point].strip())
        text = text[split_point:].strip()
    chunks.append(text)
    return chunks

def split_docx_by_rules_with_header(input_file, output_folder):
    # Tạo thư mục đích nếu chưa tồn tại
    os.makedirs(output_folder, exist_ok=True)
    
    # Đọc toàn bộ nội dung của file .docx
    doc = Document(input_file)
    full_text = [clean_text(paragraph.text) for paragraph in doc.paragraphs if clean_text(paragraph.text) != '']
    
    # Lấy thông tin về "Luật số", "Ngày ban hành", và "Tên bộ luật"
    law_number = ""
    date_info = ""
    law_title = ""
    
    for paragraph in full_text:
        if "Luật số" in paragraph:
            law_number = paragraph
        elif "ngày" in paragraph and "tháng" in paragraph and "năm" in paragraph:
            date_info = paragraph
        elif paragraph.isupper():  # Giả định rằng tiêu đề luật là viết hoa toàn bộ
            law_title = paragraph
            break
    
    # Tách văn bản thành các đoạn theo từ khóa "Điều"
    dieu_sections = []
    current_section = []
    current_condition = ""
    
    for paragraph in full_text:
        if paragraph.startswith("Điều"):
            if current_section:
                dieu_sections.append((current_condition, "\n".join(current_section)))
                current_section = []
            current_condition = paragraph
        current_section.append(paragraph)
    if current_section:
        dieu_sections.append((current_condition, "\n".join(current_section)))
    
    # Xử lý và ghi từng phần vào các file riêng biệt với thông tin bổ sung
    file_count = 0
    for i, (condition, section_text) in enumerate(dieu_sections):
        # Nếu nội dung không vượt quá kích thước tối đa, không cần chia nhỏ
        if len(section_text) <= 1024:
            output_file = os.path.join(output_folder, f"Dieu_{i+1}.txt")
            with open(output_file, 'w', encoding='utf-8') as file:
                # Thêm thông tin vào đầu file
                file.write(f"{law_number}\n")
                file.write(f"{date_info}\n\n")
                file.write(f"{law_title}\n\n")
                # Thêm phần nội dung của Điều
                file.write(section_text)
            file_count += 1
        else:
            # Nếu nội dung vượt quá kích thước tối đa, chia nhỏ và ghi vào nhiều file
            chunks = split_text_into_chunks(section_text, chunk_size=1024)
            for j, chunk in enumerate(chunks):
                output_file = os.path.join(output_folder, f"Dieu_{i+1}_{j+1}.txt")
                with open(output_file, 'w', encoding='utf-8') as file:
                    # Thêm thông tin vào đầu file
                    if j > 0:  # Chỉ bổ sung thông tin "Điều" vào file thứ 2 trở đi
                        file.write(f"{law_number}\n")
                        file.write(f"{date_info}\n\n")
                        file.write(f"{law_title}\n\n")
                        file.write(f"{condition}\n")
                    else:  # File đầu tiên không cần bổ sung thông tin "Điều"
                        file.write(f"{law_number}\n")
                        file.write(f"{date_info}\n\n")
                        file.write(f"{law_title}\n\n")
                    # Thêm phần nội dung của Điều
                    file.write(chunk)
                
                
                file_count += 1
    
    print(f"Đã tách file {input_file} thành {file_count} phần trong thư mục {output_folder}.")



DAU THÂU

In [13]:
input_file = 'D:\\Private\\DATA\\data_crawl\\dau_thau\\dau_thau.doc'
output_folder = 'D:\\Private\\DATA\\new_data\\dau_thau'

split_doc_by_rules_with_header(input_file, output_folder)

Đã tách file D:\Private\DATA\data_crawl\dau_thau\dau_thau.doc thành 216 phần trong thư mục D:\Private\DATA\new_data\dau_thau.


BO LUAT LAO DONG

In [37]:
input_file = 'D:\\Private\\DATA\\data_crawl\\lao_dong\\luat_lao_dong.docx'
output_folder = 'D:\\Private\\DATA\\new_data\\lao_dong'

split_docx_by_rules_with_header(input_file, output_folder)

Đã tách file D:\Private\DATA\data_crawl\lao_dong\luat_lao_dong.docx thành 302 phần trong thư mục D:\Private\DATA\new_data\lao_dong.


TO TUNG HINH SU

In [64]:
input_file = 'D:\\Private\\DATA\\data_crawl\\bo_luat_to_tung_hinh_su\\to_tung_hinh_su.doc'
output_folder = 'D:\\Private\\DATA\\new_data\\bo_luat_to_tung_hinh_su'

split_doc_by_rules_with_header(input_file, output_folder)

Đã tách file D:\Private\DATA\data_crawl\bo_luat_to_tung_hinh_su\to_tung_hinh_su.doc thành 755 phần trong thư mục D:\Private\DATA\new_data\bo_luat_to_tung_hinh_su.


Bo luat hang hai

In [48]:
input_file = 'D:\\Private\\DATA\\data_crawl\\bo_luat_hang_hai\\hang_hai.doc'
output_folder = 'D:\\Private\\DATA\\new_data\\bo_luat_hang_hai'

split_doc_by_rules_with_header(input_file, output_folder)

Đã tách file D:\Private\DATA\data_crawl\bo_luat_hang_hai\hang_hai.doc thành 440 phần trong thư mục D:\Private\DATA\new_data\bo_luat_hang_hai.


Bo luat hinh su


In [47]:
input_file = 'D:\\Private\\DATA\\data_crawl\\bo_luat_hinh_su\\hinh_su.doc'
output_folder = 'D:\\Private\\DATA\\new_data\\bo_luat_hinh_su'

split_doc_by_rules_with_header(input_file, output_folder)

Đã tách file D:\Private\DATA\data_crawl\bo_luat_hinh_su\hinh_su.doc thành 746 phần trong thư mục D:\Private\DATA\new_data\bo_luat_hinh_su.


Bo luat to tung dan su

In [66]:
input_file = 'D:\\Private\\DATA\\data_crawl\\bo_luat_to_tung_dan_su\\to_tung_dan_su.doc'
output_folder = 'D:\\Private\\DATA\\new_data\\bo_luat_to_tung_dan_su'

split_doc_by_rules_with_header(input_file, output_folder)

Bo Luat Dan su


In [65]:
input_file = 'D:\\Private\\DATA\\data_crawl\\bo_luat_dan_su\\dan_su_text\\bo_luat_dan_su.txt'
output_folder = 'D:\\Private\\DATA\\new_data\\bo_luat_dan_su'

split_txt_by_rules_with_header(input_file, output_folder)

Đã tách file D:\Private\DATA\data_crawl\bo_luat_dan_su\dan_su_text\bo_luat_dan_su.txt thành 413 phần trong thư mục D:\Private\DATA\new_data\bo_luat_to_tung_dan_su.
